In [2]:
import duckdb
import pandas as pd
import re
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
path_to_export = "/Users/yashwanth/Documents/GWU/Sem 3/Data Mining/Class 2/Class Material/MovieLensExport" 
duckdb.sql(f'''IMPORT DATABASE '{path_to_export}' ''')
duckdb.sql('SHOW TABLES')

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ date_dim     │
│ driver       │
│ driver2      │
│ genre_dim    │
│ links        │
│ movie_dim    │
│ movies       │
│ ratings      │
│ ratings_fact │
│ tags         │
│ user_dim     │
│ user_matrix  │
├──────────────┤
│   12 rows    │
└──────────────┘

### Let's grab all the movie titles

In [5]:
movies = duckdb.sql('SELECT * FROM movie_dim').fetchall()
titles = [m[2] for m in movies]
titles

['Toy Story',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'American President, The',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Persuasion',
 'City of Lost Children, The (Cité des enfants perdus, La)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao)',
 'Dangerous Minds',
 'Twelve Monkeys (a.k.a. 12 Monkeys)',
 'Babe',
 'Dead Man Walking',
 'It Takes Two',
 'Clueless',
 'Richard III',
 'Dead Presidents',
 'Mortal Kombat',
 'To Die For',
 'How to Make an American Quilt',
 'Seven (a.k.a. Se7en)',
 'Pocahontas',
 'When Night Is Falling',
 'Usual Suspects, The',
 'Lamerica',
 'Big Green, The',
 'Georgia',
 'Home for the Holidays',
 'Postman, The (Postino, Il)',
 'Indian in the Cupboard, The',
 'Eye for an Eye

### We will ignore case

In [7]:
lower_case = [t.lower() for t in titles]
lower_case

['toy story',
 'grumpier old men',
 'waiting to exhale',
 'father of the bride part ii',
 'heat',
 'sabrina',
 'tom and huck',
 'sudden death',
 'american president, the',
 'dracula: dead and loving it',
 'balto',
 'nixon',
 'casino',
 'sense and sensibility',
 'four rooms',
 'ace ventura: when nature calls',
 'money train',
 'get shorty',
 'copycat',
 'assassins',
 'powder',
 'leaving las vegas',
 'othello',
 'persuasion',
 'city of lost children, the (cité des enfants perdus, la)',
 'shanghai triad (yao a yao yao dao waipo qiao)',
 'dangerous minds',
 'twelve monkeys (a.k.a. 12 monkeys)',
 'babe',
 'dead man walking',
 'it takes two',
 'clueless',
 'richard iii',
 'dead presidents',
 'mortal kombat',
 'to die for',
 'how to make an american quilt',
 'seven (a.k.a. se7en)',
 'pocahontas',
 'when night is falling',
 'usual suspects, the',
 'lamerica',
 'big green, the',
 'georgia',
 'home for the holidays',
 'postman, the (postino, il)',
 'indian in the cupboard, the',
 'eye for an eye

### The simplest (though far from perfect!) way to tokenize is to just split on non-word characters using a regex

In [9]:
tokenized = [re.split('\W+', i.strip()) for i in lower_case]
tokenized_cleaned = []
for doc in tokenized:
    new_doc = [i for i in doc if i != '']
    tokenized_cleaned.append(new_doc)
tokenized_cleaned

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
/var/folders/pr/hwfrw1gn4k3bbnzlvngy93800000gn/T/ipykernel_37848/1223666479.py:1: SyntaxWarning: invalid escape sequence '\W'
  tokenized = [re.split('\W+', i.strip()) for i in lower_case]


[['toy', 'story'],
 ['grumpier', 'old', 'men'],
 ['waiting', 'to', 'exhale'],
 ['father', 'of', 'the', 'bride', 'part', 'ii'],
 ['heat'],
 ['sabrina'],
 ['tom', 'and', 'huck'],
 ['sudden', 'death'],
 ['american', 'president', 'the'],
 ['dracula', 'dead', 'and', 'loving', 'it'],
 ['balto'],
 ['nixon'],
 ['casino'],
 ['sense', 'and', 'sensibility'],
 ['four', 'rooms'],
 ['ace', 'ventura', 'when', 'nature', 'calls'],
 ['money', 'train'],
 ['get', 'shorty'],
 ['copycat'],
 ['assassins'],
 ['powder'],
 ['leaving', 'las', 'vegas'],
 ['othello'],
 ['persuasion'],
 ['city',
  'of',
  'lost',
  'children',
  'the',
  'cité',
  'des',
  'enfants',
  'perdus',
  'la'],
 ['shanghai', 'triad', 'yao', 'a', 'yao', 'yao', 'dao', 'waipo', 'qiao'],
 ['dangerous', 'minds'],
 ['twelve', 'monkeys', 'a', 'k', 'a', '12', 'monkeys'],
 ['babe'],
 ['dead', 'man', 'walking'],
 ['it', 'takes', 'two'],
 ['clueless'],
 ['richard', 'iii'],
 ['dead', 'presidents'],
 ['mortal', 'kombat'],
 ['to', 'die', 'for'],
 ['how

### Let's find all the tokens that occur in at least 3 titles and are not on our short list of stop words

In [11]:
stop_words = {'the', 'of', 'a', 's', 'and', 'in', 'to', 'on', 'i', 'for'}

doc_counts = Counter()
for doc in tokenized_cleaned:
    for token in set(doc):
        doc_counts[token]+=1

#Find the most common words
for k,v in doc_counts.most_common(10):
    print(k,v, v/len(tokenized_cleaned))

vocabulary = Counter()
example_count = 0
for k, v in doc_counts.items():
    if v>=3 and k not in stop_words:
        vocabulary[k] = v
    else:
        if example_count < 20:
            print(k,v)
        example_count +=1

print(len(doc_counts), len(vocabulary))

the 2949 0.302709915828372
of 978 0.10039006364196264
a 525 0.05389037158694313
s 405 0.04157257236707042
and 366 0.037569287620611784
in 327 0.03356600287415315
to 226 0.023198521864093616
man 186 0.01909258879080271
2 181 0.018579347156641345
on 133 0.01365222746869226
grumpier 1
to 226
exhale 1
of 978
the 2949
sabrina 2
huck 2
and 366
sudden 2
balto 1
sensibility 1
rooms 1
ventura 2
shorty 1
copycat 1
powder 2
leaving 1
othello 2
perdus 1
cité 2
9224 1979


## Now, we will create the sparse version of the features

In [13]:
featurized = []

for doc in tokenized_cleaned:
    doc_words = Counter()
    for token in doc:
        if token in vocabulary:
            doc_words[token] +=1
    featurized.append(doc_words)

featurized[:10]


[Counter({'toy': 1, 'story': 1}),
 Counter({'old': 1, 'men': 1}),
 Counter({'waiting': 1}),
 Counter({'father': 1, 'bride': 1, 'part': 1, 'ii': 1}),
 Counter({'heat': 1}),
 Counter(),
 Counter({'tom': 1}),
 Counter({'death': 1}),
 Counter({'american': 1, 'president': 1}),
 Counter({'dracula': 1, 'dead': 1, 'loving': 1, 'it': 1})]

Our quick and dirty approach here has left some movies with no features! Looking at the full title list, the missing movie title above is for Sabrina - a single-word title that was not a common enough word to meet our filter.

We will persist, just to show how this works!

In [15]:
df = pd.DataFrame(featurized)
df

,toy,story,old,men,waiting,father,bride,part,ii,heat,tom,death,american,president,dracula,dead,loving,it,nixon,casino,sense,four,ace,when,nature,calls,money,train,get,assassins,las,vegas,persuasion,city,lost,children,des,enfants,la,shanghai,dao,dangerous,minds,twelve,k,12,babe,man,walking,takes,two,richard,iii,mortal,kombat,die,how,make,an,seven,night,is,falling,big,green,home,postman,il,indian,eye,mr,don,t,be,menace,south,central,while,your,juice,hood,if,by,sea,2,beyond,french,twist,friday,from,dusk,dawn,fair,game,kicking,misérables,les,roses,bully,white,things,do,you,re,line,once,upon,time,we,were,angels,black,sheep,nick,mary,vampire,brooklyn,beautiful,girls,broken,hate,hollywood,bottle,rocket,wrong,county,nobody,loves,me,muppet,treasure,island,taxi,before,after,anne,frank,young,steal,little,boys,st,vincent,express,sam,with,disaster,neverending,pie,sky,down,year,up,close,personal,brothers,amazing,panda,adventure,diaries,13,batman,forever,beauty,day,belle,de,jour,blue,face,crimson,devil,hard,vengeance,doom,generation,first,knight,free,willy,johnny,jury,kids,living,oblivion,lord,love,human,mad,witness,nine,party,girl,safe,letter,smoke,species,strange,days,three,total,wong,everything,under,siege,dark,heroes,walk,burden,wild,bill,version,burnt,sun,rain,sunrise,billy,side,cure,castle,drop,zone,destiny,radio,dumb,eat,woman,shi,exit,eden,ed,kiss,far,adventures,yellow,dog,movie,dreams,beloved,heavy,hunted,q,interview,chronicles,just,kid,king,arthur,star,wars,...,eats,angst,auf,doc,hulk,machines,pirates,caribbean,begins,step,shaolin,juk,ray,tremors,lions,cobra,ginger,snaps,pat,inside,jeux,ralph,batoru,change,teen,follow,public,birth,passage,page,leprechaun,cooler,stuck,robert,cheaper,hyde,incident,bang,win,void,skeleton,50,non,banks,himself,unleashed,hellboy,ten,style,30,coffee,mine,chun,fail,prejudice,babylon,said,baxter,week,youth,mou,danny,robot,au,pursuit,jin,safety,college,saved,swordsman,garfield,hello,ningen,joken,softly,burgundy,maria,film,universe,prime,shark,tattoo,grudge,trinity,caesar,diggers,harvest,yours,ga,sympathy,rises,asterix,astérix,trilogy,xia,guide,ricky,suspect,nanny,gekijô,ban,pusher,beowulf,neon,evangelion,shin,kingdom,carry,revelations,obelix,obélix,yet,battlestar,galactica,ju,den,dai,upside,wizards,moving,bunny,hustle,tyler,perry,century,julia,walker,special,lane,0,elite,madagascar,march,penguins,untold,match,creep,narnia,voices,district,chappelle,trail,most,extraordinary,comic,ass,sing,raid,merry,unknown,number,musical,boots,ex,drift,museum,ant,words,busters,prison,letters,camillo,alpha,epic,think,maniac,england,librarian,limited,kevin,harrison,hitman,alvin,chipmunks,ha,rec,cloverfield,hatchet,breathe,doomsday,border,taken,futurama,rogue,frozen,zeitgeist,ip,paul,stan,fullmetal,alchemist,hangover,cloudy,chance,paranormal,activity,beasts,saga,ja,tree,fairy,gift,him,despicable,grown,expendables,j,doragon,bôru,legacy,lincoln,hare,moms,insidious,wimpy,thor,creation,kumar,need,darkest,puss,jeff,chô,senshi,sûpâ,broly,guardians,hobbit,jefferies,pain,purge,vice,kidnapping,lego,dc,sharknado,jobs,self,47,noah,maze,burr,sorry,segura,carlin,watson,note,prostokvashino,gintama,kizumonogatari
0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

So, we have a bunch of nulls, which really should be 0s.  So, let's impute zeros for these

In [17]:
df = df.fillna(0)
df

,toy,story,old,men,waiting,father,bride,part,ii,heat,tom,death,american,president,dracula,dead,loving,it,nixon,casino,sense,four,ace,when,nature,calls,money,train,get,assassins,las,vegas,persuasion,city,lost,children,des,enfants,la,shanghai,dao,dangerous,minds,twelve,k,12,babe,man,walking,takes,two,richard,iii,mortal,kombat,die,how,make,an,seven,night,is,falling,big,green,home,postman,il,indian,eye,mr,don,t,be,menace,south,central,while,your,juice,hood,if,by,sea,2,beyond,french,twist,friday,from,dusk,dawn,fair,game,kicking,misérables,les,roses,bully,white,things,do,you,re,line,once,upon,time,we,were,angels,black,sheep,nick,mary,vampire,brooklyn,beautiful,girls,broken,hate,hollywood,bottle,rocket,wrong,county,nobody,loves,me,muppet,treasure,island,taxi,before,after,anne,frank,young,steal,little,boys,st,vincent,express,sam,with,disaster,neverending,pie,sky,down,year,up,close,personal,brothers,amazing,panda,adventure,diaries,13,batman,forever,beauty,day,belle,de,jour,blue,face,crimson,devil,hard,vengeance,doom,generation,first,knight,free,willy,johnny,jury,kids,living,oblivion,lord,love,human,mad,witness,nine,party,girl,safe,letter,smoke,species,strange,days,three,total,wong,everything,under,siege,dark,heroes,walk,burden,wild,bill,version,burnt,sun,rain,sunrise,billy,side,cure,castle,drop,zone,destiny,radio,dumb,eat,woman,shi,exit,eden,ed,kiss,far,adventures,yellow,dog,movie,dreams,beloved,heavy,hunted,q,interview,chronicles,just,kid,king,arthur,star,wars,...,eats,angst,auf,doc,hulk,machines,pirates,caribbean,begins,step,shaolin,juk,ray,tremors,lions,cobra,ginger,snaps,pat,inside,jeux,ralph,batoru,change,teen,follow,public,birth,passage,page,leprechaun,cooler,stuck,robert,cheaper,hyde,incident,bang,win,void,skeleton,50,non,banks,himself,unleashed,hellboy,ten,style,30,coffee,mine,chun,fail,prejudice,babylon,said,baxter,week,youth,mou,danny,robot,au,pursuit,jin,safety,college,saved,swordsman,garfield,hello,ningen,joken,softly,burgundy,maria,film,universe,prime,shark,tattoo,grudge,trinity,caesar,diggers,harvest,yours,ga,sympathy,rises,asterix,astérix,trilogy,xia,guide,ricky,suspect,nanny,gekijô,ban,pusher,beowulf,neon,evangelion,shin,kingdom,carry,revelations,obelix,obélix,yet,battlestar,galactica,ju,den,dai,upside,wizards,moving,bunny,hustle,tyler,perry,century,julia,walker,special,lane,0,elite,madagascar,march,penguins,untold,match,creep,narnia,voices,district,chappelle,trail,most,extraordinary,comic,ass,sing,raid,merry,unknown,number,musical,boots,ex,drift,museum,ant,words,busters,prison,letters,camillo,alpha,epic,think,maniac,england,librarian,limited,kevin,harrison,hitman,alvin,chipmunks,ha,rec,cloverfield,hatchet,breathe,doomsday,border,taken,futurama,rogue,frozen,zeitgeist,ip,paul,stan,fullmetal,alchemist,hangover,cloudy,chance,paranormal,activity,beasts,saga,ja,tree,fairy,gift,him,despicable,grown,expendables,j,doragon,bôru,legacy,lincoln,hare,moms,insidious,wimpy,thor,creation,kumar,need,darkest,puss,jeff,chô,senshi,sûpâ,broly,guardians,hobbit,jefferies,pain,purge,vice,kidnapping,lego,dc,sharknado,jobs,self,47,noah,maze,burr,sorry,segura,carlin,watson,note,prostokvashino,gintama,kizumonogatari
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

OK, so now we can do a cosine similarity based on these bag-of-words names:

In [19]:
similarities = cosine_similarity(df, df)
print(similarities.shape)


def find_most_similar(distances, normalized_df, index):
    min_index = -1
    min_value = distances[index,:].max()
    max_value = -1
    max_index = -1
    for ix, val in enumerate(distances[index,:]):
        if ix != index and val < min_value:
            min_index = ix
            min_value = val
        elif ix != index and val > max_value:
            max_index = ix
            max_value = val
    print(min_index, min_value, max_index, max_value)
    return (min_index, min_value, max_index, max_value)


index = [i for i,j in enumerate(movies) if j[0]==4896][0] ## Harry Potter

min_index, min_value, max_index, max_value = find_most_similar(similarities, df, index)

print(f"Title of interest: {titles[index]}")
print(f"Least similar title : '{titles[min_index]}' with similarity {min_value} # Note: There will be a lot of movies with a 0 similarity!")
print(f"Most similar title : '{titles[max_index]}' with similarity {max_value}")


(9742, 9742)
0 0.0 4987 0.7559289460184544
Title of interest: Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone)
Least similar title : 'Toy Story' with similarity 0.0 # Note: There will be a lot of movies with a 0 similarity!
Most similar title : 'Harry Potter and the Prisoner of Azkaban' with similarity 0.7559289460184544
